# Variational AutoEncoder
**Description:** Convolutional Variational AutoEncoder (VAE) trained on CIFAR10 digits.

Autoencoders create a non continous latent space ,which dont allow interpolations in the latent space, basically it makes the latent space sparse, but as a genertaive model we need a latent space where we need a class specific compactness ,smooth
sampling such that we can create realistic represntation.
Lets take an example , if we input fish and doll as input to that model ,then from their linear interpolation in the latent space (continous) then we can get mermaid as output, but if the latent space is not continous then it can be waste as loss.

To remove this thing Variational Encoder comes to the picture which gives distributions as input to the model and enables soft encoding to the model ,where the autoencoder are very deterministic.

## Setup

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import layers

**Architecture Behind the VAE:**
The main difference in the architecture from the encoder that encoder outputs a mean(position of the distribution) and standard deviation(spread of the distribution) of each independent vector component. Now doing sampling from these vectors we generate the input for the decoder.
Z= mu+sigma*epsilon
where epsilon ~N(0,1)
though here we are going to use log variance instead of the linear variance for the making the model numerically stable. To enable the Backpropagation ,it uses the reparameterization trick in which we dissociate the sampling layer from the model.

## Create a sampling layer

In [ ]:

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


## Build the encoder

In [ ]:
latent_dim = 100

encoder_inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d_4 (Conv2D)           (None, 16, 16, 32)           896       ['input_5[0][0]']             
                                                                                                  
 conv2d_5 (Conv2D)           (None, 8, 8, 64)             18496     ['conv2d_4[0][0]']            
                                                                                                  
 conv2d_6 (Conv2D)           (None, 4, 4, 64)             36928     ['conv2d_5[0][0]']            
                                                                                            

## Build the decoder

In [ ]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(2 * 2* 64, activation="relu")(latent_inputs)
x = layers.Reshape((2, 2, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 100)]             0         
                                                                 
 dense_6 (Dense)             (None, 256)               25856     
                                                                 
 reshape_3 (Reshape)         (None, 2, 2, 64)          0         
                                                                 
 conv2d_transpose_9 (Conv2D  (None, 4, 4, 64)          36928     
 Transpose)                                                      
                                                                 
 conv2d_transpose_10 (Conv2  (None, 8, 8, 64)          36928     
 DTranspose)                                                     
                                                                 
 conv2d_transpose_11 (Conv2  (None, 16, 16, 64)        3692

## Define the VAE as a `Model` with a custom `train_step`

For the variational encoder , there are 2 losses associated with it : 1. KL Divergence loss( indicates the distance between the distributions of the features vectors, so it reduces the sparseness of the model) 2.Reconstruction loss( maintain the separibility between the different classes)

In [ ]:

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction),
                    axis=(1, 2),
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


## Train the VAE

In [ ]:
(x_train, _), (x_test, _) = keras.datasets.cifar10.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = mnist_digits.astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(mnist_digits, epochs=50, batch_size=256)

Epoch 1/50
235/235 [==============================] - 9s 21ms/step - loss: 697.3987 - reconstruction_loss: 680.4186 - kl_loss: 3.7857
Epoch 2/50
235/235 [==============================] - 5s 20ms/step - loss: 662.6628 - reconstruction_loss: 652.2198 - kl_loss: 8.3982
Epoch 3/50
235/235 [==============================] - 5s 21ms/step - loss: 655.9579 - reconstruction_loss: 645.2083 - kl_loss: 9.9399
Epoch 4/50
235/235 [==============================] - 5s 20ms/step - loss: 651.6111 - reconstruction_loss: 639.7203 - kl_loss: 10.4781
Epoch 5/50
235/235 [==============================] - 5s 20ms/step - loss: 646.5018 - reconstruction_loss: 636.3240 - kl_loss: 9.6413
Epoch 6/50
235/235 [==============================] - 5s 20ms/step - loss: 644.3215 - reconstruction_loss: 634.6954 - kl_loss: 9.3818
Epoch 7/50
235/235 [==============================] - 5s 19ms/step - loss: 643.1227 - reconstruction_loss: 633.3215 - kl_loss: 9.2152
Epoch 8/50
235/235 [==============================] - 5s 20ms